In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed, dump
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dropout, BatchNormalization, RepeatVector, MultiHeadAttention, Concatenate, Dense
from tensorflow.keras.layers import GRU, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

In [2]:
# 参数设定
threshold = 20
window = 20
slide = 1
valid_ratio = 0.2
future_label = 5
feature_cols = ['gas_mean', 'gasUsed_mean', 'counts','hour', 'minute', 'second', 'is_exist']

# 数据加载与预处理（解决 SettingWithCopyWarning）
def load_and_preprocess(filepath):
    data = pd.read_csv(filepath)
    data = data[data['isError'] == 0].copy()
    data['type'], _ = pd.factorize(data['type'])
    mask = data['type'] == 1
    data['to'] = np.where(mask, data['contractAddress'], data['to'])
    data = data.drop(columns=['type', 'contractAddress'])
    
    # 聚合数据
    data = data.groupby(['blockNumber', 'timeStamp', 'to']).agg(
        gas_mean=('gas', 'mean'),
        gasUsed_mean=('gasUsed', 'mean'),
        counts=('gas', 'size')
    ).reset_index()
    
    # 过滤低频合约
    sum_data = data.groupby('to')['counts'].sum().reset_index()
    filtered_data = data[data['to'].isin(sum_data[sum_data['counts'] > threshold]['to'])].copy()
    
    # 时间特征
    filtered_data['timeStamp'] = pd.to_datetime(filtered_data['timeStamp'], unit='s')
    filtered_data['hour'] = filtered_data['timeStamp'].dt.hour
    filtered_data['minute'] = filtered_data['timeStamp'].dt.minute
    filtered_data['second'] = filtered_data['timeStamp'].dt.second

    filtered_data['is_exist'] = 1
    return filtered_data.drop(columns=['timeStamp']).sort_values('blockNumber')

# 加载数据
filtered_data = load_and_preprocess("dataset.csv")

split_idx1 = int(0.8 * len(filtered_data))
split_idx2 = int(0.9 * len(filtered_data))

train_data = filtered_data.iloc[:split_idx1]
valid_data = filtered_data.iloc[split_idx1:split_idx2]
test_data = filtered_data.iloc[split_idx2:]

group_train_data = train_data.groupby(['to'])
group_valid_data = valid_data.groupby(['to'])
group_test_data = test_data.groupby(['to'])

In [3]:
def create_sequences(group):
    blocks = group['blockNumber'].values.astype(int)
    counts = group['counts'].values

    i = 0
    n_blocks = len(blocks)

    feature = []
    labels = []
    while i < n_blocks - 1:
        # 当前窗口的起始区块号
        start_block = blocks[i]
        end_block = start_block + window

        # 使用二分查找确定窗口结束位置
        j = np.searchsorted(blocks, end_block, side='right')

        # 窗口内实际存在的区块索引
        window_mask = (blocks >= start_block) & (blocks < end_block)
        window_blocks = blocks[window_mask]
        window_counts = counts[window_mask]

        # 填充完整的区块序列（即使没有交易）
        full_features = np.zeros((window, len(feature_cols)))
        full_counts = np.zeros(window)
        existing_indices = window_blocks - start_block
        full_features[existing_indices] = group[window_mask][feature_cols].values
        full_counts[existing_indices] = window_counts

        # 有效性检查
        valid_ratio_actual = np.sum(full_counts > 0) / window
        if valid_ratio_actual < valid_ratio:
            # 动态调整起始点：跳到下一个有效位置
            next_block_idx = j  # 默认跳到当前窗口结束位置
            if j < n_blocks:
                # 计算下一个可能有效的位置：下一个区块的前window个位置
                next_valid_start = max(blocks[j] - window + 1, start_block + 1)
                next_block_idx = np.searchsorted(blocks, next_valid_start, side='left')

            i = next_block_idx
            continue

        # 生成标签：用未来future_label个区块作为标签
        label = np.arange(future_label)
        future_start = end_block
        future_end = future_start + future_label
        future_mask = (blocks >= future_start) & (blocks < future_end)
        future_blocks = blocks[future_mask]
        future_counts = counts[future_mask]
        future_existing = future_blocks - end_block
        label[future_existing] = future_counts

        # 保存序列和标签
        feature.append(full_features)
        labels.append(label)
        # 滑动步长（至少滑动1个区块）
        i += max(slide, 1)

    return feature, labels

create_train = Parallel(n_jobs=-1)(
    delayed(create_sequences)(group) for _, group in tqdm(group_train_data, desc='Train')
)

create_valid = Parallel(n_jobs=-1)(
    delayed(create_sequences)(group) for _, group in tqdm(group_valid_data, desc='Valid')
)

create_test = Parallel(n_jobs=-1)(
    delayed(create_sequences)(group) for _, group in tqdm(group_test_data, desc='Test')
)

# 创建时间序列数据集
X_train, y_train = [], []
X_valid, y_valid = [], []
X_test, y_test = [], []
for train in create_train:
    X_train.extend(train[0])
    y_train.extend(train[1])
for valid in create_valid:
    X_valid.extend(valid[0])
    y_valid.extend(valid[1])
for test in create_test:
    X_test.extend(test[0])
    y_test.extend(test[1])

print(len(X_train), len(y_train))

Test: 100%|██████████| 799/799 [00:03<00:00, 243.58it/s]


136941 136941


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_train.shape, y_train.shape)

# 将三维数据展平为二维进行标准化
n_samples, timesteps, n_features = X_train.shape
X_train_2d = X_train.reshape(-1, n_features)
X_valid_2d = X_valid.reshape(-1, n_features)
X_test_2d = X_test.reshape(-1, n_features)
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_2d).reshape(n_samples, timesteps, n_features)
X_valid_scaled = scaler_X.transform(X_valid_2d).reshape(X_valid.shape)
X_test_scaled = scaler_X.transform(X_test_2d).reshape(X_test.shape)

# 标签标准化
y_train = np.array(y_train).reshape(-1, future_label)  # 形状变为 (n_samples, 5)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_valid_scaled = scaler_y.transform(y_valid)

print(y_train.shape)

def create_model(window, future_label, feature_dim):
    # 编码器部分
    encoder_inputs = Input(shape=(window, feature_dim))
    x = LSTM(128, return_sequences=True)(encoder_inputs)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = LSTM(64, return_sequences=True)(x)  
    x = Dropout(0.3)(x)
    encoder_outputs = x

    # 解码器部分
    decoder_input = RepeatVector(future_label)(encoder_outputs[:, -1, :])
    decoder_lstm = LSTM(64, return_sequences=True)(decoder_input)  # 解码器LSTM输出 (batch_size, 5, 64)

   # 多头注意力
    attention = MultiHeadAttention(num_heads=8, key_dim=64)(
        query=decoder_lstm, 
        value=encoder_outputs, 
        key=encoder_outputs)
    merged = Concatenate()([decoder_lstm, attention])
    merged = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(merged)
    outputs = Dense(1)(merged)
    model = Model(inputs=encoder_inputs, outputs=outputs)
    return model

# def create_model(window, future_label, feature_dim):
#     # 编码器部分（GRU替代）
#     encoder_inputs = Input(shape=(window, feature_dim))
#     x = GRU(128, return_sequences=True)(encoder_inputs)
#     x = Dropout(0.3)(x)
#     x = BatchNormalization()(x)
#     x = GRU(64, return_sequences=True)(x)  # 第二层GRU
#     x = Dropout(0.3)(x)
#     encoder_outputs = x

#     # 保持原有注意力机制结构
#     decoder_input = RepeatVector(future_label)(encoder_outputs[:, -1, :])
#     decoder_gru = GRU(64, return_sequences=True)(decoder_input)
    
#     attention = MultiHeadAttention(num_heads=4, key_dim=64)(
#         query=decoder_gru,
#         value=encoder_outputs,
#         key=encoder_outputs)
#     merged = Concatenate()([decoder_gru, attention])
#     merged = Dense(64, activation='relu', 
#                  kernel_regularizer=regularizers.l2(0.01))(merged)
    
#     outputs = Dense(1)(merged)
#     model = Model(inputs=encoder_inputs, outputs=outputs)
#     return model

# def create_model(window, future_label, feature_dim):
#     # 双向LSTM
#     encoder_inputs = Input(shape=(window, feature_dim))
#     x = Bidirectional(LSTM(128, return_sequences=True))(encoder_inputs)  # 输出维度256
#     x = Dropout(0.3)(x)
#     x = BatchNormalization()(x)
#     x = Bidirectional(LSTM(64, return_sequences=True))(x)  # 输出维度128
#     x = Dropout(0.3)(x)
#     encoder_outputs = x  # 形状(batch, timesteps, 128)

#     # 解码器部分（保持单向LSTM）
#     decoder_input = RepeatVector(future_label)(encoder_outputs[:, -1, :])  # 取最后一个时间步
#     decoder_lstm = LSTM(128, return_sequences=True)(decoder_input)  # 增大单元数以匹配编码器输出
    
#     # 调整注意力机制维度
#     attention = MultiHeadAttention(
#         num_heads=8,  # 增加注意力头数以匹配更高维度
#         key_dim=32    # 减小单个头维度防止参数膨胀
#     )(query=decoder_lstm, value=encoder_outputs, key=encoder_outputs)
    
#     merged = Concatenate()([decoder_lstm, attention])
#     merged = Dense(128, activation='swish',  # 增大维度并改用swish激活
#                  kernel_regularizer=regularizers.l1_l2(0.01, 0.01))(merged)
    
#     outputs = Dense(1)(merged)
#     model = Model(inputs=encoder_inputs, outputs=outputs)
#     return model

model = create_model(window, future_label, len(feature_cols))

step_weights = np.array([2, 1.5, 1.2, 1, 1])

def weighted_mae(y_true, y_pred):
    # 扩展权重到匹配形状
    weights = K.constant(step_weights.reshape(1, -1, 1), dtype=tf.float32)
    
    # 计算加权绝对误差
    absolute_errors = K.abs(y_true - y_pred)

    weighted_errors = weights * absolute_errors
    
    # 求平均值
    return K.mean(weighted_errors)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=weighted_mae,
    metrics=['mae']
)

# 训练模型
history = model.fit(
    X_train_scaled,
    y_train_scaled.reshape(-1, future_label, 1),  # 调整为 (samples, 5, 1)
    epochs=50,
    batch_size=64,
    validation_data=(X_valid_scaled, y_valid_scaled.reshape(-1, future_label, 1)),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=7),
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: 0.001 * (0.9 ** epoch ))
        ]  # 学习率衰减
)

# dump(scaler_X, 'scaler_X.joblib')
# dump(scaler_y, 'scaler_y.joblib')
# model.save('lstm_model')

(136941, 20, 7) (136941, 5)
(136941, 5)
Epoch 1/50
2140/2140 [==============================] - 56s 22ms/step - loss: 0.4566 - mae: 0.3233 - val_loss: 0.4564 - val_mae: 0.3479 - lr: 0.0010
Epoch 2/50
2140/2140 [==============================] - 42s 20ms/step - loss: 0.4130 - mae: 0.3154 - val_loss: 0.4470 - val_mae: 0.3415 - lr: 9.0000e-04
Epoch 3/50
2140/2140 [==============================] - 45s 21ms/step - loss: 0.4095 - mae: 0.3132 - val_loss: 0.4479 - val_mae: 0.3422 - lr: 8.1000e-04
Epoch 4/50
2140/2140 [==============================] - 44s 20ms/step - loss: 0.4076 - mae: 0.3120 - val_loss: 0.4541 - val_mae: 0.3472 - lr: 7.2900e-04
Epoch 5/50
2140/2140 [==============================] - 45s 21ms/step - loss: 0.4064 - mae: 0.3113 - val_loss: 0.4434 - val_mae: 0.3392 - lr: 6.5610e-04
Epoch 6/50
2140/2140 [==============================] - 46s 21ms/step - loss: 0.4055 - mae: 0.3106 - val_loss: 0.4491 - val_mae: 0.3437 - lr: 5.9049e-04
Epoch 7/50
2140/2140 [========================

In [5]:
def evaluate_predictions(y_true, y_pred):
    # 展平维度计算整体指标
    flat_true = y_true.flatten()
    flat_pred = y_pred.flatten()

    # 平均绝对误差和均方误差
    print(f"Overall MAE: {mean_absolute_error(flat_true, flat_pred):.2f}")
    print(f"Overall MSE: {mean_squared_error(flat_true, flat_pred):.2f}")

    # 分步长计算指标
    for step in range(future_label):
        step_mae = mean_absolute_error(y_true[:, step], y_pred[:, step])
        step_mse = mean_squared_error(y_true[:, step], y_pred[:, step])
        print(f"Step {step + 1} MAE: {step_mae:.2f}, MSE: {step_mse:.2f}")

y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(
    y_pred_scaled.reshape(-1, future_label))  # 形状恢复为 (n_samples, 5)
print(y_test.shape, y_pred.shape)

evaluate_predictions(y_test, y_pred)

524/524 [==============================] - 5s 7ms/step
(16753, 5) (16753, 5)
Overall MAE: 0.97
Overall MSE: 5.56
Step 1 MAE: 1.06, MSE: 5.82
Step 2 MAE: 0.76, MSE: 5.50
Step 3 MAE: 0.77, MSE: 4.87
Step 4 MAE: 0.97, MSE: 5.29
Step 5 MAE: 1.27, MSE: 6.34
